<a href="https://colab.research.google.com/github/TheDerebeyi/HousePricesRegressionProject/blob/main/RegressionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
# Importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbs

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate
import sklearn.metrics

from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
import sklearn.tree
from sklearn.neighbors import KNeighborsRegressor

In [69]:
#reading dataset from csv file

df = pd.read_csv("Melbourne_housing_FULL.csv")
df_copy = df.copy()

df_copy.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [70]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [71]:
print(f"Shape: {df_copy.shape}")
print(f"Size: {df_copy.size}")
print(f"No of cols: {df_copy.shape[1]}")

Shape: (34857, 21)
Size: 731997
No of cols: 21


In [72]:
df_copy.duplicated().sum() # checking if there is any duplicated rows

1

In [73]:
df_copy.loc[df_copy.duplicated(keep=False),:] #searching the duplicated rows

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
15857,Nunawading,1/7 Lilian St,3,t,NaN,SP,Jellis,17/06/2017,15.4,3131.0,...,3.0,2.0,405.0,226.0,2000.0,Manningham City Council,-37.82678,145.16777,Eastern Metropolitan,4973.0
15858,Nunawading,1/7 Lilian St,3,t,NaN,SP,Jellis,17/06/2017,15.4,3131.0,...,3.0,2.0,405.0,226.0,2000.0,Manningham City Council,-37.82678,145.16777,Eastern Metropolitan,4973.0


In [74]:
df_copy = df_copy.drop(15857, axis="index") #dropping one of the dups

In [75]:
df_copy.index = range(len(df_copy)) #reindexing

In [76]:
df_copy.duplicated().sum()

0

In [77]:
df_copy.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Price             2871
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom2            15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       740
YearBuilt          160
CouncilArea         33
Lattitude        13402
Longtitude       14524
Regionname           8
Propertycount      342
dtype: int64

In [78]:
df_copy.tail()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
34851,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34852,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34853,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34854,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0
34855,Yarraville,3 Tarrengower St,2,h,1020000.0,PI,RW,24/02/2018,6.3,3013.0,...,1.0,0.0,250.0,103.0,1930.0,Maribyrnong City Council,-37.81810,144.89351,Western Metropolitan,6543.0


In [79]:
for i in range(len(df_copy)): #There was 34009 discrete address value, that means every row has its own unique adress value more or less.
  cell = df_copy.at[i, 'Address'] #This is a problem. We can see if we filter out street numbers, there are common street names.
  index = -1
  for j in range(len(cell)): #Finding first space character
    if(cell[j] == ' '):
      index = j
      break
  df_copy.at[i, 'Address'] = df_copy.at[i, 'Address'][index+1:]

df_copy.tail()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
34851,Yarraville,Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34852,Yarraville,Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34853,Yarraville,Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34854,Yarraville,Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0
34855,Yarraville,Tarrengower St,2,h,1020000.0,PI,RW,24/02/2018,6.3,3013.0,...,1.0,0.0,250.0,103.0,1930.0,Maribyrnong City Council,-37.81810,144.89351,Western Metropolitan,6543.0


In [80]:
df_copy.rename(columns={"Address":"Street"}, inplace = True)
df_copy.tail()

,Suburb,Street,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
34851,Yarraville,Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34852,Yarraville,Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34853,Yarraville,Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34854,Yarraville,Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0
34855,Yarraville,Tarrengower St,2,h,1020000.0,PI,RW,24/02/2018,6.3,3013.0,...,1.0,0.0,250.0,103.0,1930.0,Maribyrnong City Council,-37.81810,144.89351,Western Metropolitan,6543.0


In [96]:
df_copy.nunique()

Suburb             351
Street           10845
Rooms               12
Price             2872
Distance           216
                 ...  
Bathroom_6.0         2
Bathroom_7.0         2
Bathroom_8.0         2
Bathroom_9.0         2
Bathroom_12.0        2
Length: 563, dtype: int64

In [82]:
meanLandsize = np.mean(df_copy.Landsize)
stdLandsize = np.std(df_copy.Landsize)
meanBuildingarea = np.mean(df_copy.BuildingArea)
stdBuildingarea = np.std(df_copy.BuildingArea)

In [83]:
for i in range(len(df_copy.Landsize)): #replacing outliers using z score
  landsize = df_copy.at[i, 'Landsize']
  z = abs(landsize-meanLandsize)/stdLandsize
  if (z > 3):
    df_copy.at[i, 'Landsize'] = meanLandsize

for i in range(len(df_copy.BuildingArea)):
  landsize = df_copy.at[i, 'BuildingArea']
  z = abs(landsize-meanBuildingarea)/stdBuildingarea
  if (z > 3):
    df_copy.at[i, 'BuildingArea'] = meanBuildingarea

In [84]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34856 entries, 0 to 34855
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34856 non-null  object 
 1   Street         34856 non-null  object 
 2   Rooms          34856 non-null  int64  
 3   Type           34856 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34856 non-null  object 
 6   SellerG        34856 non-null  object 
 7   Date           34856 non-null  object 
 8   Distance       34855 non-null  float64
 9   Postcode       34855 non-null  float64
 10  Bedroom2       26639 non-null  float64
 11  Bathroom       26630 non-null  float64
 12  Car            26128 non-null  float64
 13  Landsize       23046 non-null  float64
 14  BuildingArea   13741 non-null  float64
 15  YearBuilt      15550 non-null  float64
 16  CouncilArea    34853 non-null  object 
 17  Lattitude      26880 non-null  float64
 18  Longti

In [85]:
df_copy.tail()

,Suburb,Street,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
34851,Yarraville,Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34852,Yarraville,Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34853,Yarraville,Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34854,Yarraville,Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0
34855,Yarraville,Tarrengower St,2,h,1020000.0,PI,RW,24/02/2018,6.3,3013.0,...,1.0,0.0,250.0,103.0,1930.0,Maribyrnong City Council,-37.81810,144.89351,Western Metropolitan,6543.0


In [86]:
df_copy.isna().sum()

Suburb               0
Street               0
Rooms                0
Type                 0
Price             7609
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
dtype: int64

In [87]:
df_copy["Car"] = df_copy["Car"].fillna(df_copy["Car"].mode().iloc[0])    #replacing null values
df_copy["Bedroom2"] = df_copy["Bedroom2"].fillna(df_copy["Bedroom2"].mode().iloc[0])
df_copy["Bathroom"] = df_copy["Bathroom"].fillna(df_copy["Bathroom"].mode().iloc[0])
df_copy["YearBuilt"]= df_copy["YearBuilt"].fillna(df_copy["YearBuilt"].mode().iloc[0])
df_copy["CouncilArea"]= df_copy["CouncilArea"].fillna(df_copy["CouncilArea"].mode().iloc[0])
df_copy["Regionname"]= df_copy["Regionname"].fillna(df_copy["Regionname"].mode().iloc[0])
df_copy["Propertycount"]= df_copy["Propertycount"].fillna(df_copy["Propertycount"].mode().iloc[0])
df_copy["Price"] = df_copy["Price"].fillna(df_copy["Price"].mean())
df_copy["Landsize"] = df_copy["Landsize"].fillna(df_copy["Landsize"].mean())
df_copy["BuildingArea"] = df_copy["BuildingArea"].fillna(df_copy["BuildingArea"].mean())
df_copy["Distance"] = df_copy["Distance"].fillna(df_copy["Distance"].mean())
df_copy["Postcode"] = df_copy["Postcode"].fillna(df_copy["Postcode"].mean())
df_copy["Lattitude"] = df_copy["Lattitude"].fillna(df_copy["Lattitude"].mean())
df_copy["Longtitude"] = df_copy["Longtitude"].fillna(df_copy["Longtitude"].mean())

In [88]:
df_copy.isna().sum()

Suburb           0
Street           0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
YearBuilt        0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64

In [89]:
categoricalVars = ["Type","Method","SellerG","Date","CouncilArea","Regionname", "Car", "Bedroom2", "Bathroom"]
numVars = ["Rooms", "Price", "Distance", "Postcode", "Landsize", "BuildingArea", "YearBuilt", "Lattitude", "Longtitude", "Propertycount"]

In [90]:
def one_hot_encoder(dataframe, categorical_col, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_col, drop_first=drop_first)
    return dataframe

df_copy = one_hot_encoder(df_copy, categoricalVars, drop_first=True)

label_encoder = LabelEncoder()

df_copy["Street"] = label_encoder.fit_transform(df_copy["Street"]) #using one hot encoder isn't very useful in our case for Street and Suburb cols
df_copy["Suburb"] = label_encoder.fit_transform(df_copy["Suburb"])

In [91]:
scaler = StandardScaler()
df_copy[numVars] = scaler.fit_transform(df_copy[numVars])

In [92]:
df_copy.tail()

,Suburb,Street,Rooms,Price,Distance,Postcode,Landsize,BuildingArea,YearBuilt,Lattitude,...,Bathroom_1.0,Bathroom_2.0,Bathroom_3.0,Bathroom_4.0,Bathroom_5.0,Bathroom_6.0,Bathroom_7.0,Bathroom_8.0,Bathroom_9.0,Bathroom_12.0
34851,348,1523,0.999024,0.757890,-0.719544,-0.945333,0.167390,0.000000,0.083952,1.307964e-03,...,1,0,0,0,0,0,0,0,0,0
34852,348,7187,-1.062974,-0.285952,-0.719544,-0.945333,-0.919668,-0.918635,2.000773,-6.150763e-02,...,0,1,0,0,0,0,0,0,0,0
34853,348,8908,-1.062974,-0.608626,-0.719544,-0.945333,-0.651747,-0.629443,1.281965,-1.542174e-01,...,1,0,0,0,0,0,0,0,0,0
34854,348,9264,-0.031975,0.158387,-0.719544,-0.945333,0.000000,0.000000,0.083952,-8.962483e-14,...,1,0,0,0,0,0,0,0,0,0
34855,348,9563,-1.062974,-0.053203,-0.719544,-0.945333,-0.585865,-0.936709,-1.513399,-9.417678e-02,...,1,0,0,0,0,0,0,0,0,0


In [93]:
models = {'Lasso': {
'model': Lasso()
},
'LinearRegression': {
'model': LinearRegression()
},
'Ridge': {
'model': Ridge()
},
'ElasticNet': {
'model': ElasticNet()
},
'KNeighborsRegressor': {
'model': KNeighborsRegressor()
},
'RandomForestRegressor': {
'model': RandomForestRegressor()
},
'GradientBoostingRegressor': {
'model': GradientBoostingRegressor()
},
'AdaBoostRegressor':  {
'model': AdaBoostRegressor(n_estimators = 5, learning_rate = 1.2, loss = 'exponential', random_state = 2)
}}

models

{'Lasso': {'model': Lasso()},
 'LinearRegression': {'model': LinearRegression()},
 'Ridge': {'model': Ridge()},
 'ElasticNet': {'model': ElasticNet()},
 'KNeighborsRegressor': {'model': KNeighborsRegressor()},
 'RandomForestRegressor': {'model': RandomForestRegressor()},
 'GradientBoostingRegressor': {'model': GradientBoostingRegressor()},
 'AdaBoostRegressor': {'model': AdaBoostRegressor(learning_rate=1.2, loss='exponential', n_estimators=5,
                    random_state=2)}}

In [94]:
y = df_copy["Price"]
X = df_copy.drop("Price", axis=1)

In [95]:
for name in models.keys():
    model = models[name]['model']
    cv_results = cross_validate(model, X, y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_root_mean_squared_error', 'r2'])
    print(f'########## {name} ##########')
    print(f'MAE: {round(cv_results["test_neg_mean_absolute_error"].mean(), 4)}')
    print(f'MSE: {round(cv_results["test_neg_mean_squared_error"].mean(), 4)}')
    print(f'RMSE: {round(cv_results["test_neg_root_mean_squared_error"].mean(), 4)}')
    print(f'R2: {round(cv_results["test_r2"].mean(), 4)}')

########## Lasso ##########
MAE: -0.6355
MSE: -0.9871
RMSE: -0.9913
R2: 0.0066
########## LinearRegression ##########
MAE: -6321537.1371
MSE: -220390385615763.72
RMSE: -7689052.2009
R2: -241236550224771.7
########## Ridge ##########
MAE: -0.4713
MSE: -0.5411
RMSE: -0.7342
R2: 0.4546
########## ElasticNet ##########
MAE: -0.6362
MSE: -0.9869
RMSE: -0.9912
R2: 0.0067
########## KNeighborsRegressor ##########
MAE: -0.6587
MSE: -0.9773
RMSE: -0.9863
R2: 0.0166
########## RandomForestRegressor ##########
MAE: -0.3974
MSE: -0.4665
RMSE: -0.6807
R2: 0.532
########## GradientBoostingRegressor ##########
MAE: -0.4352
MSE: -0.4884
RMSE: -0.697
R2: 0.5091
########## AdaBoostRegressor ##########
MAE: -0.6017
MSE: -0.7291
RMSE: -0.8521
R2: 0.2636
